# Exact Diagonalization of a 1D Spin Chain
<b> Christina Lee

Category: Grad

Prerequisites: Many-Body Quantum Mechanics </b>

# Table of Contents
 <p><div class="lev1"><a href="#Exact-Diagonalization-of-a-1D-Spin-Chain-1">Exact Diagonalization of a 1D Spin Chain</a></div><div class="lev2"><a href="#Theoretical-Background-1.1">Theoretical Background</a></div><div class="lev2"><a href="#Initializing-Variables-1.2">Initializing Variables</a></div><div class="lev2"><a href="#Creating-Basis-Vectors-1.3">Creating Basis Vectors</a></div><div class="lev3"><a href="#Conservation-of-Magnetization-1.3.1">Conservation of Magnetization</a></div><div class="lev2"><a href="#The-Hamiltonian-Operator-1.4">The Hamiltonian Operator</a></div><div class="lev2"><a href="#For-Further-Information-1.5">For Further Information</a></div>

## Theoretical Background 
\begin{equation}
\mathcal{H}= J \sum\limits_{\langle i,j \rangle} \mathbf{S}_i \cdot \mathbf{S}_j
\end{equation}
\begin{equation}
\mathcal{H} = J \sum\limits_{\langle i,j \rangle}\left[ S_i^z S_j^z + \frac{1}{2} \left(S^+_iS^-_j + S^-_i S^+_j \right)\right]
\end{equation}

\begin{equation}
\mathcal{H}|\Psi (...,i,j,...) \rangle 
    = \left( \sum\limits_{\langle i,j \rangle} S^z_i S^z_j \right) | \Psi (...,i,j,...) \rangle +
    \sum\limits_{\langle i,j \rangle} 
\end{equation}

In [1]:
# Pkg.update()
# Pkg.add("Iterators")
# Pkg.add("PyPlot")
using Iterators
using PyPlot

## Initializing Variables

In [12]:
n=3
nstates=2^n

8

## Creating Basis Vectors

If we were doing this with a different language, we could actually represent our ones and zeros with individual bits instead of the collection of 8-bits in our `Int8`. But right now lets do a trade off of computational efficiency versus coding efficiency and call this implementation sufficient.

In [18]:
Ψv=collect(product(repeated(convert(Int8,0):convert(Int8,1),n)...))

8-element Array{Tuple{Int8,Int8,Int8},1}:
 (0,0,0)
 (1,0,0)
 (0,1,0)
 (1,1,0)
 (0,0,1)
 (1,0,1)
 (0,1,1)
 (1,1,1)

### Conservation of Magnetization

If we seperate our Hilbert Space into subspaces of conserved quantities, like total magnetization, that shrinks the size of our problem.  Here I create a vector `m` which holds the total magnetization for each of our basis states.  
    
We can then use a Julia sorting function to sort both `m` and `Ψv` into blocks of constant magnetization.

In [20]:
m=zeros(Int64,nstates)
for i in 1:nstates
    m[i]=sum(Ψv[i])
end
ind=sortperm(m)
m=m[ind]
Ψv=Ψv[ind]

8-element Array{Tuple{Int8,Int8,Int8},1}:
 (0,0,0)
 (1,0,0)
 (0,1,0)
 (0,0,1)
 (1,1,0)
 (1,0,1)
 (0,1,1)
 (1,1,1)

## The Hamiltonian Operator

In [22]:
M=zeros(nstates,nstates)

for j=1:nstates
	diag=0
	for i=1:(n-1)
        if Ψv[j][i]==Ψv[j][i+1]
			diag+=.25
		else
			diag-=.25
		end
	end
    M[j,j]=diag
end

In [14]:
M

8x8 Array{Float64,2}:
 0.5  0.0   0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0  -0.5  0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0  0.0  -0.5  0.0  0.0
 0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0
 0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.5

In [ ]:
function GetOperator(basis,n)
	n2=2^n
	M=zeros(n2,n2)
    maxbin=length(bin(n2-1))

	for j=1:n2

        for k=1:(n2/2+1)
            count='2'
            if j!=k
                for m=1:maxbin
                    if basis[j][m]!=basis[k][m] && count=='2'
                        count=basis[j][m]
                    elseif basis[j][m]!=basis[k][m] && basis[j][m]!=count && count !='2'
                        count='3'
                        M[j,k]=.5
                        M[k,j]=.5
                    elseif basis[j][m]!=basis[k][m] && count=='3'
                        M[j,k]=0
                        M[k,j]=0
                    end
                end

            end
        end
	end
	return M
end


## For Further Information
* http://physics.bu.edu/~sandvik/vietri/dia.pdf
